# 07 - Test and Deploy Training Pipeline to Vertex AI Pipelines

The purpose of this notebook is to compile and run the TFX pipeline to AI Platform Managed Pipelines. The notebook covers the following tasks:
1. Test the pipeline locally using local runner.
2. Set the pipeline deployment configuration.
3. Build Container Image
4. Compile TFX Pipeline
5. Submit a pipeline job to AI Platform Pipelines (Managed).

## Setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import kfp
import tfx

print("Tensorflow Version:", tfx.__version__)
print("KFP Version:", kfp.__version__)

In [ ]:
!rm -r src/raw_schema/.ipynb_checkpoints/

In [ ]:
PROJECT = 'ksalama-cloudml' # Change to your project Id.
REGION = 'us-central1'
BUCKET = 'ksalama-cloudml-us' # Change to your bucket.
REPO_URL = "https://github.com/ksalama/ucaip-labs.git" # Change to your github repo.
BRANCH = "simplified"

DATASET_DISPLAY_NAME = 'chicago_taxi_tips'
MODEL_DISPLAY_NAME = 'chicago_taxi_tips_classifier_v1'

CICD_IMAGE_NAME = 'cicd:latest'
CICD_IMAGE_URI = f"gcr.io/{PROJECT}/{CICD_IMAGE_NAME}"

## Build CI/CD  Container Image for Cloud Build

This is the runtime environment where the steps of testing and deploying the model will be executed.

In [ ]:
!echo $CICD_IMAGE_URI

In [ ]:
!gcloud builds submit --tag $CICD_IMAGE_URI build/. --timeout=15m

## 1. Run the Pipeline CICD steps locally

### Set pipeline configurations for the local run

In [ ]:
os.environ["DATASET_DISPLAY_NAME"] = DATASET_DISPLAY_NAME
os.environ["MODEL_DISPLAY_NAME"]  =  MODEL_DISPLAY_NAME
os.environ["PROJECT"] = PROJECT
os.environ["REGION"] = REGION
os.environ["GCS_LOCATION"] = f"gs://{BUCKET}/ucaip_demo/chicago_taxi/pipelines/e2e_tests"
os.environ["TRAIN_LIMIT"] = "1000"
os.environ["TEST_LIMIT"] = "100"
os.environ["UPLOAD_MODEL"] = "0"
os.environ["ACCURACY_THRESHOLD"] = "0.1"
os.environ["BEAM_RUNNER"] = "DirectRunner"
os.environ["TRAINING_RUNNER"] = "local"

In [ ]:
from src.pipelines import config
for key, value in config.__dict__.items():
    if key.isupper(): print(f'{key}: {value}')

### Run e2e pipeline test

In [ ]:
!py.test src/tests/pipeline_deployment_tests.py::test_e2e_pipeline -s

### Set the pipeline configurations for the Vertex AI run

In [ ]:
VERSION = 'tfx-0-30'
os.environ["DATASET_DISPLAY_NAME"] = DATASET_DISPLAY_NAME
os.environ["PIPELINE_NAME"] = f'chicago_taxi_tips_train_pipeline_{VERSION}'
os.environ["PROJECT"] = PROJECT
os.environ["REGION"] = REGION
os.environ["GCS_LOCATION"] = f"gs://{BUCKET}/ucaip_demo/chicago_taxi/pipelines_managed_runner"
os.environ["TRAIN_LIMIT"] = "85000"
os.environ["TEST_LIMIT"] = "15000"
os.environ["BEAM_RUNNER"] = "DataflowRunner"
os.environ["TRAINING_RUNNER"] = "caip"
os.environ["TFX_IMAGE_URI"] = f"gcr.io/{PROJECT}/chicago_taxi_tips:{VERSION}"

In [ ]:
from src.pipelines import config

import importlib
importlib.reload(config)

for key, value in config.__dict__.items():
    if key.isupper(): print(f'{key}: {value}')

### Build container image

This is the tfx runtime environment for the training pipeline steps.

In [ ]:
!echo $TFX_IMAGE_URI

In [ ]:
!gcloud builds submit --tag $TFX_IMAGE_URI . --timeout=15m

### Compile pipeline

In [ ]:
from src.pipelines import runner

pipeline_definition_file = f'{config.PIPELINE_NAME}.json'
pipeline_definition = runner.compile_pipeline(pipeline_definition_file)
# pipeline_definition

### Submit run to AI Platform Managed Pipelines

In [ ]:
from kfp.v2.google.client import AIPlatformClient

pipeline_client = AIPlatformClient(
    project_id=PROJECT, region=REGION)
                 
pipeline_client.create_run_from_job_spec(
    job_spec_path=pipeline_definition_file,
    parameter_values={
        'learning_rate': 0.003,
        'batch_size': 512,
        'hidden_units': '128,128',
        'num_epochs': 30,
    }
)

## 2. Execute the Model Deployment CI/CD rountine in Cloud Build

In [ ]:
GCS_LOCATION = f"gs://{BUCKET}/ucaip_demo/chicago_taxi/pipelines"
TEST_GCS_LOCATION = f"gs://{BUCKET}/ucaip_demo/chicago_taxi/pipelines/e2e_tests"
TRAIN_LIMIT = 1000
TEST_LIMIT = 100
UPLOAD_MODEL = 0
ACCURACY_THRESHOLD = 0.1
BEAM_RUNNER = "DataflowRunner"
TRAINING_RUNNER = "vertex"
VERSION = 'tfx-0-30'
PIPELINE_NAME = f'chicago_taxi_tips_train_pipeline_{VERSION}'
PIPELINES_STORE = os.path.join(GCS_LOCATION, "compiled_pipelines")

TFX_IMAGE_URI = f"gcr.io/{PROJECT}/chicago_taxi_tips:{VERSION}"

SUBSTITUTIONS=f"""\
_REPO_URL='{REPO_URL}',\
_BRANCH={BRANCH},\
_CICD_IMAGE_URI={CICD_IMAGE_URI},\
_PROJECT={PROJECT},\
_REGION={REGION},\
_GCS_LOCATION={GCS_LOCATION},\
_TEST_GCS_LOCATION={TEST_GCS_LOCATION},\
_DATASET_DISPLAY_NAME={DATASET_DISPLAY_NAME},\
_MODEL_DISPLAY_NAME={MODEL_DISPLAY_NAME},\
_TRAIN_LIMIT={TRAIN_LIMIT},\
_TEST_LIMIT={TEST_LIMIT},\
_UPLOAD_MODEL={UPLOAD_MODEL},\
_ACCURACY_THRESHOLD={ACCURACY_THRESHOLD},\
_BEAM_RUNNER={BEAM_RUNNER},\
_TRAINING_RUNNER={TRAINING_RUNNER},\
_TFX_IMAGE_URI={TFX_IMAGE_URI},\
_PIPELINE_NAME={PIPELINE_NAME},\
_PIPELINES_STORE={PIPELINES_STORE}\
"""

!echo $SUBSTITUTIONS

In [ ]:
!gcloud builds submit --no-source --timeout=60m --config build/pipeline-deployment.yaml --substitutions {SUBSTITUTIONS} --machine-type=e2-highcpu-8